## 默认查询参数

声明不属于路径参数的其他函数参数时，它们将被自动解释为"查询字符串"参数

查询字符串是键值对的集合，这些键值对位于 URL 的 ？ 之后，并以 & 符号分隔。

例如，在以下 url 中：

http://127.0.0.1:8000/items/?skip=0&limit=10

查询参数为：

skip：对应的值为   0
limit：对应的值为   10
由于它们是 URL 的一部分，因此它们的"原始值"是字符串。

但是，当你为它们声明了 Python 类型（在上面的示例中为 int）时，它们将转换为该类型并针对该类型进行校验。

In [2]:
import uvicorn
from fastapi import FastAPI

app = FastAPI()

fake_items_db = [{"item_name": "Foo"}, {"item_name": "Bar"}, {"item_name": "Baz"}]


@app.get("/items/")
async def read_item(skip: int = 0, limit: int = 10):
    return fake_items_db[skip : skip + limit]

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [20824]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:54845 - "GET /items/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [20824]


## 必需查询参数

In [ ]:
当你为非路径参数声明了默认值时（目前而言，我们所知道的仅有查询参数），则该参数不是必需的。

如果你不想添加一个特定的值，而只是想使该参数成为可选的，则将默认值设置为 None。

但当你想让一个查询参数成为必需的，不声明任何默认值就可以：

In [4]:
import uvicorn
from fastapi import FastAPI

app = FastAPI()


@app.get("/items/{item_id}")
async def read_user_item(item_id: str, needy: str):
    item = {"item_id": item_id, "needy": needy}
    return item

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [20824]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:55313 - "GET /items/4?needy=yes HTTP/1.1" 200 OK
INFO:     127.0.0.1:55358 - "GET /items/4 HTTP/1.1" 422 Unprocessable Entity


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [20824]


这里的查询参数 needy 是类型为 str 的必需查询参数。

浏览器输入 http://127.0.0.1:8009/items/4?needy=yes

会看{"item_id":"4","needy":"yes"}

如果浏览器输入 http://127.0.0.1:8009/items/4

会报错 {"detail":[{"type":"missing","loc":["query","needy"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.5/v/missing"}]}"}

## 可选查询参数

通过同样的方式，你可以将它们的默认值设置为 None 来声明可选查询参数：
在这个例子中，函数参数 q 将是可选的，并且默认值为 None。

In [3]:
import uvicorn
from typing import Union
from fastapi import FastAPI

app = FastAPI()

@app.get("/items/{item_id}")
async def read_item(item_id: str, q: Union[str, None] = None):
    if q:
        return {"item_id": item_id, "q": q}
    return {"item_id": item_id}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [20824]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:55114 - "GET /items/4 HTTP/1.1" 200 OK
INFO:     127.0.0.1:55137 - "GET /items/4?q=hello HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [20824]


浏览器输入 http://127.0.0.1:8009/items/4

会看到 {"item_id":"4"}

输入 http://127.0.0.1:8009/items/4?q=hello

会看到 {"item_id":"4","q":"hello"}

当然，你也可以定义一些参数为必需的，一些具有默认值，而某些则完全是可选的：

In [ ]:
import uvicorn
from typing import Union
from fastapi import FastAPI
app = FastAPI()
@app.get("/items/{item_id}")
async def read_user_item(
    item_id: str, needy: str, skip: int = 0, limit: Union[int, None] = None
):
    item = {"item_id": item_id, "needy": needy, "skip": skip, "limit": limit}
    return item

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

在这个例子中，有3个查询参数：

needy，一个必需的 str 类型参数。

skip，一个默认值为 0 的 int 类参数。
。
limit，一个可选的 in类型参数。 数

你还可以像在 路径参数 中那样使用numn。


## 多个路径和查询参数

你可以同时声明多个路径参数和查询参数，FastAPI 能够识别它们。

而且你不需要以任何特定的顺序来声明。

它们将通过名称被检测到：

In [6]:
import uvicorn
from typing import Union
from fastapi import FastAPI
app = FastAPI()
@app.get("/users/{user_id}/items/{item_id}")
async def read_user_item(
    user_id: int, item_id: str, q: Union[str, None] = None, short: bool = False
):
    item = {"item_id": item_id, "owner_id": user_id}
    if q:
        item.update({"q": q})
    if not short:
        item.update(
            {"description": "This is an amazing item that has a long description"}
        )
    return item

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [20824]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:55415 - "GET /users/3/items/4?q=foo&short=yes HTTP/1.1" 200 OK
INFO:     127.0.0.1:55433 - "GET /users/3/items/4?q=foo HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [20824]


浏览器输入 http://127.0.0.1:8009/users/3/items/4?q=foo&short=yes

会看到 {"item_id":"4","owner_id":3,"q":"foo"}

浏览器输入 http://127.0.0.1:8009/users/3/items/4?q=foo

会看到 {"item_id":"4","owner_id":3,"q":"foo","description":"This is an amazing item that has a long description"}